In [ ]:
# inputs A = (60000 x 10 x P) and biases B = (60000 x 10) for each digit
# labels: 60000 x 10


In [1]:
# import libraries

import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp

In [2]:
from gurobipy import Model, GRB


In [10]:
model = Model("convex_linearized_model")
#sam's comment
#for now
n = 60000 #number of training examples
l = 10 #number of labels
z = model.addVars(60000, 10, vtype=GRB.CONTINUOUS, name="myVar")
w = []  # List to store the created variables``

for i in range(p):
    var = model.addVar(vtype=GRB.CONTINUOUS, name=f"weight_{i}")
    w.append(var) # the only actual decision variables, aka the model parameters

z = np.zeros(n,l) #stores the outputs of the linearized model
for i in range(n):
    for j in range(l):
        var = model.addVar(vtype=GRB.CONTINUOUS, name=f"z{i}_{j}")
        z[i,j] = var
        model.addConstrs(z[i,j] = sum(A[i,j,k]*w[k] for k in range(p)) + B[i,j] for j in range(10))# z = Aw + B

exp_z = np.zeros(n,l)# exp_z = exp(z)

for i in range(n):
    for j in range(l):
        var = model.addVar(vtype=GRB.CONTINUOUS, name=f"exp{i}_{j}")
        exp_z[i,j] = var
        model.addGenConstrExp(z[i,j], exp_z[i,j])

denominators = np.zeros(n) #denominators = sum(exp_z)
soft_maxed = np.zeros(n,l) #soft_maxed = exp_z/denominators
for i in range(n):
    model.addVar(vtype=GRB.CONTINUOUS, name=f"denom{i}")
    denominators[i] = var
    model.addConstr(var = sum(exp_z[i,j] for j in range(l)))
    for j in range(l):
        var = model.addVar(vtype=GRB.CONTINUOUS, name=f"soft_maxed{i}_{j}")
        soft_maxed[i,j] = var
        model.addConstr(var * demominators[i] = exp_z[i,j])

nll = np.zeros(n) #nll = -log(soft_maxed)
for i in range(n):
    j = labels[i]
    var = model.addVar(vtype=GRB.CONTINUOUS, name=f"nll{i}")
    nll[i] = var
    model.addGenConstrLog(soft_maxed[i,j], nll[i])

model.setObjective(sum(nll[i] for i in range(n)), GRB.MAXIMIZE)
